# Lab 1. Forecasting Agent

## はじめに

このノートブックでは、[Amazon Bedrock Agents](https://aws.amazon.com/bedrock/agents/) で最初のサブエージェントを作成する方法を説明します。

Amazon Bedrock Agents を使用すると、生成 AI アプリケーションで自然言語を使用して複数ステップのビジネスタスクを実行できます。

最初の例では、予測エージェントを作成します。顧客はエージェントに現在のエネルギー消費量とその予測に関する情報を返すように依頼できます。

以下は、このモジュール上に構築されるアーキテクチャの一部を表しています。

![予測エージェントのアーキテクチャ](img/forecast_agent.png)

この例では、エージェントがコード解釈機能を使用して、エネルギー使用量とその予測データに基づいて基本的な計算を実行できるようにします。また、[Amazon Bedrock Knowledge Bases](https://aws.amazon.com/bedrock/knowledge-bases/) を使用して、予測モデルとその機能に関するドキュメントも提供しています。

完了の理由から、エネルギー予測は、SageMakerでホストされているMLモデルを使用して、このエージェントの範囲外ですでに行われているものと仮定します。

## セットアップ

まず、pip から boto3 の依存関係をインストールします。完全な機能を使用するには、最新バージョンであることを確認してください。

In [ ]:
!pip uninstall boto3 botocore awscli --yes

In [ ]:
# Install latest boto3
!python3 -m pip install --force-reinstall --no-cache -q --no-dependencies -r ../requirements.txt

#### カーネルを再起動

最新のマルチエージェント機能を適用する際に問題が発生する場合は、この行のコメントを解除してカーネルを再起動し、パッケージの更新が有効になるようにします

In [ ]:
import IPython

# IPython.Application.instance().kernel.do_shutdown(True)

boto3のバージョンを確認してください

In [ ]:
!pip freeze | grep boto3

## エージェントの作成

このセクションでは、ノートブック全体でヘルパーとして機能するグローバル変数を宣言し、最初のエージェントの作成を開始します。

In [ ]:
import boto3
import os
import json
import time

sts_client = boto3.client('sts')
session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name
account_id_suffix = account_id[:3]
agent_suffix = f"{region}-{account_id_suffix}"

s3_client = boto3.client('s3', region)
bedrock_client = boto3.client('bedrock-runtime', region)

agent_foundation_model = [
    'anthropic.claude-3-5-sonnet-20240620-v1:0',
    'anthropic.claude-3-sonnet-20240229-v1:0',
    'anthropic.claude-3-haiku-20240307-v1:0'
]

In [ ]:
forecast_agent_name = f"forecast-{agent_suffix}"

forecast_lambda_name = f"fn-forecast-agent-{agent_suffix}"

forecast_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{forecast_agent_name}'

dynamodb_table = f"{forecast_agent_name}-table"
dynamodb_pk = "customer_id"
dynamodb_sk = "day"

dynamoDB_args = [dynamodb_table, dynamodb_pk, dynamodb_sk]

knowledge_base_name = f'{forecast_agent_name}-kb'
suffix = f"{region}-{account_id}"

knowledge_base_description = "KB containing information on how forecasting process is done"
bucket_name = f'forecast-agent-kb-{suffix}'


### ヘルパー関数のインポート

次のセクションでは、`bedrock_agent_helper.py` と `knowledge_base_helper` を Python パスに追加して、ファイルが認識され、その機能を呼び出すことができるようにします。

次に、ヘルパー クラス `bedrock_agent_helper.py` と `knowledge_base_helper.py` をインポートします。

これらのファイルには、ラボをスムーズに実行することに重点を置いたヘルパー クラスが含まれています。

Bedrock とのすべてのやり取りは、これらのクラスによって処理されます。

このラボで呼び出すメソッドは次のとおりです:

`agents.py` の場合:
- `create_agent`: 新しいエージェントとそれぞれの IAM ロールを作成します
- `add_action_group_with_lambda`: Lambda 関数を作成し、以前に作成したエージェントのアクション グループとして追加します
- `create_agent_alias`: このエージェントのエイリアスを作成します
- `invoke`: エージェントを実行します

`knowledge_bases.py` の場合:
- `create_or_retrieve_knowledge_base`: Amazon Bedrock にナレッジ ベースが存在しない場合は作成するか、以前に作成したナレッジ ベースに関する情報を取得します。
- `synchronize_data`: S3 上のファイルを読み取り、テキスト情報をベクトルに変換して、その情報を Vector Database に追加します。

In [ ]:
import sys

sys.path.insert(0, ".")
sys.path.insert(1, "..")

from utils.bedrock_agent_helper import (
    AgentsForAmazonBedrock
)
from utils.knowledge_base_helper import (
    KnowledgeBasesForAmazonBedrock
)
agents = AgentsForAmazonBedrock()
kb = KnowledgeBasesForAmazonBedrock()

## ナレッジベースを作成して同期する

このセクションでは、Amazon Bedrock ナレッジベースを作成し、そこにデータを取り込みます。

このデータには、予測プロセスの実行方法に関する基本情報が含まれています。

**この作成プロセスには数分かかる場合があります。**

In [ ]:
%%time
kb_id, ds_id = kb.create_or_retrieve_knowledge_base(
    knowledge_base_name,
    knowledge_base_description,
    bucket_name
)

print(f"Knowledge Base ID: {kb_id}")
print(f"Data Source ID: {ds_id}")

## S3 にロードする合成データを作成する

他の場所でデータを取得する代わりに、Amazon Bedrock の LLM を使用してデータを生成します。
生成されるこの偽のデータは、S3 バケットにアップロードされ、Amazon Bedrock ナレッジベースに追加されます。

In [ ]:
path = "kb_documents"

# Check whether the specified path exists or not
is_exist = os.path.exists(path)
if not is_exist:
   # Create a n ew directory if it does not exist
   os.makedirs(path)
   print("The {} directory was created!".format(path))
else:
   print("The {} directory already exists!".format(path))

Bedrock 上で LLM を呼び出し、Python を使用してローカル ファイルを書き込むヘルパー メソッドを作成する

In [ ]:
def invoke_bedrock_generate_energy_files(prompt):
    message_list = []

    initial_message = {
        "role": "user",
        "content": [
            {
                "text": prompt
            }
        ],
    }

    message_list.append(initial_message)

    response = bedrock_client.converse(
        modelId=agent_foundation_model[0],
        messages=message_list,
        inferenceConfig={
            "maxTokens": 2048,
            "temperature": 0
        },
    )

    return response['output']['message']


def write_file(file_name, content):
    f = open(file_name, 'w')
    f.write(content)
    f.close()

### データ プロンプトの生成
LLM モデルを使用して予測情報を含む 1 つのファイルを生成

In [ ]:
text_generation_energy_instructions = '''
    You will be act as data-scientist that knows how to do machine learning
    forecasting using Python and scikit learn. You will generate a step-by-step
    on how to create a forecast process for a time-series data.

    This data has the following json structure:
    {
        "customer_id": "1",
        "day": "2024/06/01",
        "sumPowerReading": "120.0",
        "kind":"measured"
    }

    Choose one forecast algorithm, that works on scikit-learn, explain the
    details on how to create a step-by-step forecast, with code sample,
    showcasing how to run forecast on this data.

    Include some explanation on how to understand the forecasted values and
    how to decide the factors driving those values.

    Answer only with the step-by-step, avoid answer with afirmations like:
    "OK, I can generate it," or "Yes, please find following example."
    Be direct and only reply the step-by-step.
'''

solar_energy_file_name = 'forecasting-info.txt'

response_message = invoke_bedrock_generate_energy_files(
    text_generation_energy_instructions
)

print("Generated data to be stored in the KB:\n", response_message['content'][0]['text'])
write_file(
    '{}/{}'.format(path, solar_energy_file_name),
    response_message['content'][0]['text']
)

### s3 へのデータのアップロード
生成されたファイルを Amazon S3 バケットにアップロードします。

In [ ]:
def upload_directory(path, bucket_name):
    for root,dirs,files in os.walk(path):
        for file in files:
            file_to_upload = os.path.join(root,file)
            print(f"uploading file {file_to_upload} to {bucket_name}")
            s3_client.upload_file(file_to_upload,bucket_name,file)

### ナレッジベースの同期
データが s3 バケットで利用できるようになったので、ナレッジベースに同期してみましょう。

In [ ]:
upload_directory("kb_documents", bucket_name)

# sync knowledge base
kb.synchronize_data(kb_id, ds_id)

## エージェントの作成

予測プロセスの実行方法に関する情報を含む `Amazon Bedrock ナレッジベース` と、ユーザー リクエストを処理する `アクション グループ`、消費量の増加などの基本的なユーザー リクエストを計算する `コード解釈` 機能を備えた予測エージェントを作成します。

正確なエージェントを作成するには、エージェントが実行すべきことと実行すべきでないことについて明確な指示を設定することが重要です。エージェントが使用可能なナレッジベースとアクション グループをいつ使用すべきかを明確に定義することも重要です。

エージェントに次の指示を提供します:
```
あなたは、顧客がエネルギー消費パターンと将来の使用予測を理解するのを支援するエネルギー アシスタントです。

次のような能力が必要です:
1. 過去のエネルギー消費量を分析する
2. 消費量を予測する
3. 使用統計を生成する
4. 特定の顧客の予測を更新する

コアとなる行動:
1. 顧客に詳細を尋ねる前に、常に利用可能な情報システムを使用する
2. プロフェッショナルでありながら会話的な口調を維持する
3. 社内システムやデータ ソースを参照せずに、明確で直接的な回答を提供する
4. 情報をわかりやすい方法で提示する
5. オンザフライ計算にはコード生成機能と解釈機能を使用する。自分で計算しようとしないでください。
6. グラフをプロットしないでください。ユーザーから求められた場合は、プロットを拒否してください。代わりに、データの概要を提供します

応答スタイル:
- 役に立ち、ソリューション指向であること
- 明確で非技術的な言語を使用すること
- 実用的な洞察を提供することに重点を置くこと
- 自然な会話の流れを維持すること
- 簡潔でありながら有益であること
- ユーザーに必要のない余分な情報を追加しないこと
```

また、予測方法の説明のためのナレッジ ベースを次の手順で接続します:
```
特定の予測生成方法を説明する必要がある場合は、このナレッジ ベースにアクセスします。
```

また、エージェントが次のツールを使用できるようにします:
- `get_forecasted_consumption`: 今後 3 か月のエネルギー使用量予測を取得します
- `get_historical_consumption`: 現在までのエネルギー使用量履歴を取得します
- `get_consumption_statistics`: 現在の月の使用状況分析を取得します
- `update_forecasting`: 特定の月のエネルギー予測を更新します

In [ ]:
kb_info = kb.get_kb(kb_id)
kb_arn = kb_info['knowledgeBase']['knowledgeBaseArn']

In [ ]:
kb_config = {
    'kb_id': kb_id,
    'kb_instruction': """Access this knowledge base when needing to explain specific forecast generation methodology."""
}

In [ ]:
agent_description = """You are a energy usage forecast bot.
You can retrieve historical energy consumption, forecasted consumption, usage statistics and update a forecast for a specific user"""

agent_instruction = """You are an Energy Assistant that helps customers understand their energy consumption patterns and future usage expectations.

Your capabilities include:
1. Analyzing historical energy consumption
2. Providing consumption forecasts
3. Generating usage statistics
4. Updating forecasts for specific customers

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. Use code generation and interpretation capabilities for any on the fly calculation. DO NOT try to calculate things by yourself.
6. DO NOT plot graphs. Refuse to do so when asked by the user. Instead provide an overview of the data

Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user"""

forecast_agent = agents.create_agent(
    forecast_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    kb_arns=[kb_arn],
    code_interpretation=True
)

forecast_agent

### ナレッジベースの関連付け
エージェントを作成したので、先ほど作成したナレッジベースをエージェントに関連付けましょう。

In [ ]:
agents.associate_kb_with_agent(
    forecast_agent[0],
    kb_config['kb_instruction'],
    kb_config['kb_id']
)

### Lambda の作成

エージェントがタスクを実行できるようにするには、タスク実行を実装する AWS Lambda 関数を作成します。次に、この Lambda 関数をエージェントのアクション グループに提供します。アクション グループを使用してエージェントが実行できるアクションを定義する方法の詳細については、[こちら](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html) を参照してください。

このブロックでは、Lambda 関数コードを生成します。

In [ ]:
%%writefile forecast.py
import boto3
import json
import os

from boto3.dynamodb.conditions import Key, Attr
from datetime import datetime
from decimal import Decimal

dynamodb_resource = boto3.resource('dynamodb')
dynamodb_table = os.getenv('dynamodb_table')
dynamodb_pk = os.getenv('dynamodb_pk')
dynamodb_sk = os.getenv('dynamodb_sk')
truncated_month = datetime.today().replace(day=1, hour=0, minute=0, second=0, microsecond=0)


def get_named_parameter(event, name):
    return next(item for item in event['parameters'] if item['name'] == name)['value']
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def trunc_datetime(month,year):
    return datetime.today().replace(year =int(year), month=int(month), day=1, hour=0, minute=0, second=0, microsecond=0)

def put_dynamodb(table_name, item):
    table = dynamodb_resource.Table(table_name)
    resp = table.put_item(Item=item)
    return resp

def read_dynamodb(
    table_name: str, 
    pk_field: str,
    pk_value: str,
    sk_field: str=None, 
    sk_value: str=None,
    attr_key: str=None,
    attr_val: str=None
):
    try:

        table = dynamodb_resource.Table(table_name)
        # Create expression
        if sk_field:
            key_expression = Key(pk_field).eq(pk_value) & Key(sk_field).eq(sk_value)
        else:
            key_expression = Key(pk_field).eq(pk_value)

        if attr_key:
            attr_expression = Attr(attr_key).eq(attr_val)
            query_data = table.query(
                KeyConditionExpression=key_expression,
                FilterExpression=attr_expression
            )
        else:
            query_data = table.query(
                KeyConditionExpression=key_expression
            )
        
        return query_data['Items']
    except Exception:
        print(f'Error querying table: {table_name}.')

def get_forecasted_consumption(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="kind", attr_val="forecasted")

def get_historical_consumption(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         attr_key="kind", attr_val="measured")

def get_consumption_statistics(customer_id):
    return read_dynamodb(dynamodb_table, 
                         dynamodb_pk, 
                         customer_id, 
                         dynamodb_sk, 
                         truncated_month.strftime('%Y/%m/%d'))

def update_forecasting(customer_id, month, year, usage):
    current_date = trunc_datetime(month, year)
    if  current_date >= truncated_month:
        item = {
            'customer_id': customer_id,
            'day': current_date.strftime('%Y/%m/%d'),
            'sumPowerReading': Decimal(usage),
            'kind': 'forecasted'
        }
        put_dynamodb(dynamodb_table, item)
        return "Day: {} updated for customer: {}".format(current_date.strftime('%Y/%m/%d'), customer_id)
    else:
        return "You're trying to change a past date: {} for customer: {}, which is not allowed".format(current_date.strftime('%Y/%m/%d'), customer_id)

def lambda_handler(event, context):
    print(event)
    
    # name of the function that should be invoked
    function = event.get('function', '')

    # parameters to invoke function with
    parameters = event.get('parameters', [])
    customer_id = get_named_parameter(event, "customer_id")

    if function == 'get_forecasted_consumption':
        result = get_forecasted_consumption(customer_id)
    elif function == 'get_historical_consumption':
        result = get_historical_consumption(customer_id)
    elif function == 'get_consumption_statistics':
        result = get_consumption_statistics(customer_id)
    elif function == 'update_forecasting':
        month = get_named_parameter(event, "month")
        year = get_named_parameter(event, "year")
        usage = get_named_parameter(event, "usage")
        result = update_forecasting(customer_id, month, year, usage)
    else:
        result = f"Error, function '{function}' not recognized"

    response = populate_function_response(event, result)
    print(response)
    return response

### 利用可能なアクションの定義

次に、[関数の詳細](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html) を使用して、エージェントが実行できる利用可能なアクションを定義します。このタスクは OpenAPI スキーマを使用して実行することもできます。これは、アプリケーションですでに OpenAPI スキーマを使用できる場合に非常に便利です。

関数の詳細を作成するときは、関数とそのパラメータについて明確な説明を提供することが重要です。エージェントは、実行するタスクを正しく調整するためにそれらに依存しているためです。

In [ ]:
functions_def = [
    {
        "name": "get_forecasted_consumption",
        "description": """Gets the next 3 months energy usage forecast""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_historical_consumption",
        "description": """Gets energy usage history to date""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "get_consumption_statistics",
        "description": """Gets current month usage analytics""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            }
        }
    },
    {
        "name": "update_forecasting",
        "description": """Updates the energy forecast for a specific month""",
        "parameters": {
            "customer_id": {
                "description": "Unique customer identifier",
                "required": True,
                "type": "string"
            },
            "month": {
                "description": "Target update month. In the format MM",
                "required": True,
                "type": "integer"
            },
            "year": {
                "description": "Target update year. In the format YYYY",
                "required": True,
                "type": "integer"
            },
            "usage": {
                "description": "New consumption value",
                "required": True,
                "type": "integer"
            }
        }
    }
]

### アクション グループの作成とエージェントへのアタッチ
ここで、この Lambda 関数と関数の詳細をこのエージェントのアクション グループとして追加し、準備します。

In [ ]:
agents.add_action_group_with_lambda(
    agent_name=forecast_agent_name,
    lambda_function_name=forecast_lambda_name,
    source_code_file="forecast.py",
    agent_functions=functions_def,
    agent_action_group_name="forecast_consumption_actions",
    agent_action_group_description="Function to get usage forecast for a user ",
    dynamo_args=dynamoDB_args
)

## DynamoDB へのデータのロード

エージェントを作成したので、生成されたデータを DynamoDB にロードしましょう。これにより、エージェントはライブ データとやり取りしてアクションを実行できるようになります。

In [ ]:
with open("1_user_sample_data.json") as f:
    table_items = [json.loads(line) for line in f]

agents.load_dynamodb(dynamodb_table, table_items)

データが DynamoDB にロードされたことをテストする

In [ ]:
resp = agents.query_dynamodb(
    dynamodb_table, dynamodb_pk, '1', dynamodb_sk, "2024/06/01"
)
resp

## エージェントのテスト

では、作成したエージェントが動作するか確認するために、テスト エイリアス「TSTALIASID」を使用します。これにより、エージェントのドラフト バージョンを呼び出すことができます。

### 予測取得のテスト
まず予測取得アクションをテストしましょう

In [ ]:
%%time
response = agents.invoke(
    """can you give me my forecasted energy consumption? 
    How does it compare with my past energy usage? My customer id is 1""", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

### 過去の消費量を取得するテスト
これで、過去のエネルギー消費量をテストし、コード解釈を使用して夏季の平均エネルギー消費量を計算できるようになりました

In [ ]:
%%time
response = agents.invoke(
    "can you give me my past energy consumption? What is my average spending on summer months? My customer id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

### ナレッジベースへのアクセスをテストする
予測アルゴリズムに関する質問をして、ナレッジベースへのアクセスを確認しましょう

In [ ]:
%%time
response = agents.invoke(
    "What's algorithm used for forecast?", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

### 予測更新のテスト
これで、予測を更新する機能をテストできます

In [ ]:
%%time
response = agents.invoke(
    "Can you update my forecast for month 12/2024? I will be travelling and my estimate will be 50. My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

### 予測が更新されたことを確認する
予測を更新したら、予測が更新されたことを確認し、新しいグラフをプロットしましょう

In [ ]:
%%time
response = agents.invoke(
    "Can you give me my forecasted energy consumption month by month? My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

In [ ]:
time.sleep(60)

### 予測統計のテスト
最後に、統計取得機能をテストしましょう

In [ ]:
%%time
response = agents.invoke(
    "can you give me my current consumption? My id is 1", 
    forecast_agent[0], enable_trace=True
)
print("====================")
print(response)

## エイリアスの作成

ご覧のとおり、エージェントを `TSTALIASID` とともに使用してタスクを完了できます。
ただし、マルチエージェント コラボレーションの場合は、最初にエージェントをテストし、完全に機能するようになってからのみ使用することが想定されます。
したがって、マルチエージェント コラボレーションでエージェントをサブエージェントとして使用するには、まずエージェント エイリアスを作成し、それを新しいバージョンに接続する必要があります。

エージェントをテストして検証したので、次にそのエイリアスを作成しましょう。

In [ ]:
forecast_agent_alias_id, forecast_agent_alias_arn = agents.create_agent_alias(
    forecast_agent[0], 'v1'
)

## 情報の保存
次のノートブックで使用する環境変数をいくつか保存しましょう。

In [ ]:
forecast_agent_arn = agents.get_agent_arn_by_name(forecast_agent_name)
forecast_agent_id = forecast_agent[0]
forecast_kb = knowledge_base_name
forecast_dynamodb = dynamodb_table

%store forecast_agent_arn
%store forecast_agent_alias_arn
%store forecast_agent_alias_id
%store forecast_lambda_name
%store forecast_agent_name
%store forecast_agent_id
%store forecast_kb
%store forecast_dynamodb

In [ ]:
forecast_agent_arn, forecast_agent_alias_arn, forecast_agent_alias_id

## 次のステップ
おめでとうございます！これで予測エージェントが作成されました。次はソーラーパネルエージェントを作成します